In [1]:
import requests
import pandas as pd
import numpy as np
import os
import matplotlib
import nltk
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ast

In [2]:
""" TIME SERIES WITH #VIDEOS, LIKES, COMMENTS """

class EDA_Analysis():

    def __init__(self) -> None:
        def data_read_csv(path, title):
            video = path + "/" + title
            df_ = pd.read_csv(video)

            return df_

        path = "/Users/jesustellez/Desktop/aiDynamics/API Data extraction/Data"
        df_comments = data_read_csv(path, "comments_eeuu.csv")
        self.df_comments = df_comments
        df_videos = data_read_csv(path, "videos_eeuu.csv")
        df_videos["Date"] = df_videos["publishedAt"].str.split("T", expand=True)[0]
        self.df_videos = df_videos
        df_calendar = data_read_csv(path, "calendar.csv")
        df_calendar["Week-Year"] = df_calendar["Week"].astype(str) + "-" + df_calendar["Year"].astype(str)
        self.df_calendar = df_calendar

    def __returndata__(self):
        return self.df_videos, self.df_calendar, self.df_comments
    
    def EDA_timely(self, period):

        df_ts_analysis = self.df_videos.merge(
            self.df_calendar[["Date", "Day", "Week","Month", "Month-Year", "Week-Year"]], 
            on="Date", 
            how="left")
        
        df_ts_analysis_wy = df_ts_analysis.groupby(period, as_index=False).agg({"viewCount":"sum", "video_id":"count","commentCount":"sum"})
        return df_ts_analysis_wy

    def EDA_time_graph(self, period, fields, df_analysis):

        fig = go.Figure()
        for i in fields:
            fig.add_trace(go.Scatter(x=df_analysis[period], y=df_analysis[i]))

        fig.show()

df_videos, df_calendar, df_comments = EDA_Analysis().__returndata__()

#df_analysis = EDA_Analysis().EDA_timely(period ="Day")
#EDA_Analysis().EDA_time_graph("Day", ["commentCount"], df_analysis)

"""
THIS CODE IS FOR BETTER PLOTS

fig = make_subplots(specs=[[{"secondary_y": True}]])

#fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ts_analysis["Week-Year"], y=df_ts_analysis["viewCount"]),secondary_y=False)
fig.add_trace(go.Scatter(x=df_ts_analysis["Week-Year"], y=df_ts_analysis["video_id"]), secondary_y=True)
fig.add_trace(go.Scatter(x=df_ts_analysis["Week-Year"], y=df_ts_analysis["commentCount"]),secondary_y=True)
fig.show()

"""



'\nTHIS CODE IS FOR BETTER PLOTS\n\nfig = make_subplots(specs=[[{"secondary_y": True}]])\n\n#fig = go.Figure()\nfig.add_trace(go.Scatter(x=df_ts_analysis["Week-Year"], y=df_ts_analysis["viewCount"]),secondary_y=False)\nfig.add_trace(go.Scatter(x=df_ts_analysis["Week-Year"], y=df_ts_analysis["video_id"]), secondary_y=True)\nfig.add_trace(go.Scatter(x=df_ts_analysis["Week-Year"], y=df_ts_analysis["commentCount"]),secondary_y=True)\nfig.show()\n\n'

In [3]:
df_comments_eda = (
    df_comments.groupby("author_name", as_index=False)
    .agg(
        text_display = pd.NamedAgg(column='test_display', aggfunc='count'),
        likes_sum = pd.NamedAgg(column='likes_count', aggfunc='sum'),
        likes_max = pd.NamedAgg(column='likes_count', aggfunc='max'),
        likes_min = pd.NamedAgg(column='likes_count', aggfunc='min')
        )
)
df_comments_eda["mean_likes"] = df_comments_eda["likes_sum"]/df_comments_eda["text_display"]


In [14]:
df_comments_eda = df_comments_eda.sort_values(by=["text_display"]).loc[df_comments_eda["mean_likes"]>10]

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_comments_eda["author_name"], y=df_comments_eda["mean_likes"]))
fig.add_trace(go.Scatter(x=df_comments_eda["author_name"], y=df_comments_eda["likes_max"]))
fig.add_trace(go.Scatter(x=df_comments_eda["author_name"], y=df_comments_eda["likes_min"]))
fig.show()

In [5]:
df_comments_eda = df_comments_eda.sort_values(by=["text_display"]).loc[df_comments_eda["text_display"]>5]
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_comments_eda["author_name"], y=df_comments_eda["text_display"]))
fig.show()

In [6]:
import plotly.express as px
fig = px.scatter(df_comments_eda.loc[df_comments_eda["text_display"]>0].loc[df_comments_eda["mean_likes"]>0], x="mean_likes", y="text_display", hover_data = ["author_name"]
                 ) #size='mean_likes'
fig.show()

In [7]:
df_comments_eda

,author_name,text_display,likes_sum,likes_max,likes_min,mean_likes
2610,Jesus Says,6,86.0,25.0,10.0,14.333333


In [8]:
aux_list = list()
df_videos_ = df_videos[["video_id","tags"]]
df_videos_["tags"] = df_videos.tags.str.split(",")
df_tags_analysis = df_videos_.explode("tags")

df_tags_analysis = df_tags_analysis.merge(df_videos[["video_id","viewCount","likeCount","commentCount"]], on="video_id", how="left")

/var/folders/8d/87cwf9891qb19jpvx9ms8v000000gn/T/ipykernel_43586/3554473749.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
df_tags_analysis["tags"] = df_tags_analysis["tags"].str.replace("['[]]", "")
df_tags_analysis = df_tags_analysis.drop_duplicates()
df_tags_analysis = df_tags_analysis.groupby('tags', as_index=False).agg(
    view_sum = pd.NamedAgg(column='viewCount', aggfunc='sum'),
    like_sum = pd.NamedAgg(column='likeCount', aggfunc='sum'),
    comment_sum = pd.NamedAgg(column='commentCount', aggfunc='sum')
)

/var/folders/8d/87cwf9891qb19jpvx9ms8v000000gn/T/ipykernel_43586/4074999250.py:1: FutureWarning:

The default value of regex will change from True to False in a future version.



In [10]:
df_tags_analysis = df_tags_analysis.loc[df_tags_analysis["comment_sum"]>500].sort_values(by=["like_sum"])
fig = go.Figure()
fig.add_trace(go.Bar(x=df_tags_analysis["tags"], y=df_tags_analysis["like_sum"]))
fig.show()

In [11]:
df_comments_analysis = df_comments[["video_id", "author_name","text_original", "likes_count", "date_published"]]
df_comments_analysis = (
    df_videos[["video_id", "Date", "viewCount","likeCount","commentCount"]]
    .merge(
        df_comments_analysis,
        on="video_id",
        how="left"
    )
)

df_comments_analysis[["date_published", "Date"]] = df_comments_analysis[["date_published", "Date"]].apply(pd.to_datetime)

df_comments_analysis= df_comments_analysis.dropna()

df_comments_analysis["diffDays"] = (df_comments_analysis["date_published"] - df_comments_analysis["Date"]).dt.days
display(df_comments_analysis)

,video_id,Date,viewCount,likeCount,commentCount,author_name,text_original,likes_count,date_published,diffDays
0,w-NueyHZi0M,2022-08-26,5822,195,137.0,K Bear,FBI is going to have a bad day soon. Midterm h...,0.0,2022-08-27,1
1,w-NueyHZi0M,2022-08-26,5822,195,137.0,Greg Giles,"The Documents are now witnesses, Hollywood cou...",0.0,2022-08-27,1
2,w-NueyHZi0M,2022-08-26,5822,195,137.0,Peter Gatliff,One story that may relavent to this story. Tru...,0.0,2022-08-27,1
3,w-NueyHZi0M,2022-08-26,5822,195,137.0,Erik Jehle,Trump will be Indicted.,0.0,2022-08-27,1
4,w-NueyHZi0M,2022-08-26,5822,195,137.0,Suz Firth,𝐓𝐫𝐮𝐦𝐩 𝐢𝐬 𝐚 𝐝𝐮𝐦𝐛 𝐓𝐇𝐈𝐄𝐅.,2.0,2022-08-27,1
...,...,...,...,...,...,...,...,...,...,...
478716,GusQdzkg1sI,2022-01-03,8580,29,9.0,Burt Lade,"There is no such thing as ""free"" Covid tests. ...",0.0,2022-01-12,9
478717,GusQdzkg1sI,2022-01-03,8580,29,9.0,Edward Carberry,Why anyone would want to Damage their Blood Br...,0.0,2022-01-07,4
478718,GusQdzkg1sI,2022-01-03,8580,29,9.0,Rose Smith,Apparently in Florida the Governor has them no...,2.0,2022-01-07,4
478719,GusQdzkg1sI,2022-01-03,8580,29,9.0,Cometome 819,Thank you for posting great video.,1.0,2022-01-05,2


In [12]:
df_comments_analysis_days = (df_comments_analysis.groupby(["diffDays"], as_index=False)
    .agg(likes_sum = pd.NamedAgg(column='likes_count', aggfunc='sum'),
    comments_count = pd.NamedAgg(column='text_original', aggfunc='count')
    )
)

fig = go.Figure()
fig.add_trace(go.Bar(x=df_comments_analysis_days["diffDays"], y=df_comments_analysis_days["comments_count"]))
fig.show()

In [13]:
df_videos

,Unnamed: 0,video_id,channeltitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,Date
0,0,w-NueyHZi0M,Washington Post,You don’t need to be a spy to break the Espion...,The warrant authorizing the search of former p...,"['1917 espionage act', '1918 sedition act', 'N...",2022-08-26T20:35:05Z,5822,195,NaN,137.0,P,2022-08-26
1,1,Q8dH8f6svEA,Washington Post,WATCH: White House holds news conference,White House press secretary Karine Jean-Pierre...,"['breaking news', 'breaking news video', 'joe ...",2022-08-26T18:31:57Z,13707,260,NaN,38.0,P,2022-08-26
2,2,mXXsWBvEJLQ,Washington Post,Jerome Powell: Fighting inflation will cause ‘...,Federal Reserve Chair Jerome H. Powell deliver...,"['breaking news', 'breaking news video', 'live...",2022-08-26T14:13:41Z,11365,173,NaN,48.0,P,2022-08-26
3,3,OTRYIcD4738,Washington Post,Why Biden acted on student loans,President Biden has drawn the ire of activists...,"['News', 'The Washington Post', 'WaPo Video', ...",2022-08-26T14:11:27Z,52398,397,NaN,1534.0,P,2022-08-26
4,4,ktClzhOodkA,Washington Post,Democrats keep refusing to back a Biden 2024 run,More than a dozen congressional Democrats have...,"['2022', '2022 midterm elections', '2022 midte...",2022-08-25T21:25:27Z,6588,71,NaN,85.0,P,2022-08-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40370,40370,GusQdzkg1sI,Washington Post,Rapid covid tests: How they work and where to ...,Here’s what you need to know about these at-ho...,"['News', 'The Washington Post', 'WaPo Video', ...",2022-01-03T17:04:43Z,8580,29,NaN,9.0,P,2022-01-03
40371,40371,ogvnEoRbJro,Washington Post,Colorado governor holds a news conference abou...,"Colorado governor Jared Polis (D), and Boulder...","['breaking news', 'breaking news video', 'clim...",2021-12-31T17:50:10Z,36803,218,NaN,NaN,P,2021-12-31
40372,40372,GusQdzkg1sI,Washington Post,Rapid covid tests: How they work and where to ...,Here’s what you need to know about these at-ho...,"['News', 'The Washington Post', 'WaPo Video', ...",2022-01-03T17:04:43Z,8580,29,NaN,9.0,P,2022-01-03
40373,40373,ogvnEoRbJro,Washington Post,Colorado governor holds a news conference abou...,"Colorado governor Jared Polis (D), and Boulder...","['breaking news', 'breaking news video', 'clim...",2021-12-31T17:50:10Z,36803,218,NaN,NaN,P,2021-12-31
